# Pitfall 04: 弱工具变量的诊断与处理

## 🎯 学习目标

完成本 notebook 后，你将能够：

1. **理解** 弱工具变量问题的本质和后果
2. **诊断** 工具变量是否为弱工具（F 统计量规则）
3. **应用** 弱工具变量的稳健推断方法
4. **识别** 工具变量的排斥性假设违背

---

## 📖 背景故事

> **场景：教育回报率估计**
>
> 你想估计多受一年教育对收入的影响。但教育是内生的（能力高的人既上更多学又赚更多钱）。
>
> 你找到一个工具变量：出生季度。理论上，出生季度影响入学年龄，从而影响受教育年限，
> 但不直接影响收入。
>
> 但分析结果很奇怪：
> - 估计值不稳定（每次差别很大）
> - 置信区间异常宽
> - 有时候甚至是负数（教育降低收入？）
>
> **这就是弱工具变量问题！本 notebook 将帮你诊断和解决。**

---

## Part 1: IV 方法回顾

### 1.1 为什么需要工具变量？

当处理变量 $D$ 与误差项 $\epsilon$ 相关时，OLS 有偏。

$$Y = \beta D + \epsilon, \quad Cov(D, \epsilon) \neq 0$$

### 1.2 工具变量条件

工具变量 $Z$ 需要满足：

1. **相关性 (Relevance)**: $Cov(Z, D) \neq 0$
2. **排斥性 (Exclusion)**: $Cov(Z, \epsilon) = 0$（Z 只通过 D 影响 Y）

### 1.3 2SLS 估计

**第一阶段**: $D = \pi Z + X\gamma + v$

**第二阶段**: $Y = \beta \hat{D} + X\delta + u$

### 1.4 弱工具变量问题

当 $Cov(Z, D)$ 很小（接近 0）时：
- 第一阶段 $\hat{D}$ 不精确
- 第二阶段估计方差爆炸
- 即使样本量很大，估计仍有偏

In [ ]:
# 环境准备
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from linearmodels.iv import IV2SLS
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = ['Arial Unicode MS', 'SimHei', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 环境准备完成")

In [ ]:
def generate_iv_data(n=1000, beta_true=0.5, iv_strength='strong', seed=42):
    """
    生成 IV 模拟数据
    
    Args:
        n: 样本量
        beta_true: 真实处理效应
        iv_strength: 工具变量强度 ('strong', 'moderate', 'weak', 'very_weak')
    """
    np.random.seed(seed)
    
    # 工具变量强度参数
    strength_params = {
        'strong': 0.5,      # π = 0.5
        'moderate': 0.2,    # π = 0.2
        'weak': 0.1,        # π = 0.1
        'very_weak': 0.03   # π = 0.03
    }
    pi = strength_params[iv_strength]
    
    # 未观测混淆变量
    U = np.random.randn(n)
    
    # 工具变量（与 U 独立）
    Z = np.random.randn(n)
    
    # 处理变量（受 Z 和 U 影响）
    D = pi * Z + 0.5 * U + np.random.randn(n) * 0.5
    
    # 结果变量（受 D 和 U 影响）
    Y = beta_true * D + 0.8 * U + np.random.randn(n) * 0.5
    
    # 计算理论 F 统计量
    # F ≈ n * π^2 / var(v)
    theoretical_F = n * pi**2 / 0.25  # 简化估计
    
    return pd.DataFrame({
        'Y': Y,
        'D': D,
        'Z': Z
    }), theoretical_F

# 生成不同强度的数据
data_strong, F_strong = generate_iv_data(iv_strength='strong')
data_moderate, F_moderate = generate_iv_data(iv_strength='moderate')
data_weak, F_weak = generate_iv_data(iv_strength='weak')
data_very_weak, F_very_weak = generate_iv_data(iv_strength='very_weak')

print("生成不同 IV 强度的数据：")
print(f"  强 IV: 理论 F ≈ {F_strong:.1f}")
print(f"  中等 IV: 理论 F ≈ {F_moderate:.1f}")
print(f"  弱 IV: 理论 F ≈ {F_weak:.1f}")
print(f"  极弱 IV: 理论 F ≈ {F_very_weak:.1f}")

---

## Part 2: 弱工具变量诊断

### 2.1 第一阶段 F 统计量

**经验法则** (Stock & Yogo, 2005):
- F > 10: 工具变量足够强
- F < 10: 可能是弱工具变量

In [ ]:
def first_stage_diagnostics(df):
    """
    第一阶段诊断
    """
    Y, D, Z = df['Y'], df['D'], df['Z']
    
    # 第一阶段回归: D ~ Z
    Z_with_const = sm.add_constant(Z)
    first_stage = sm.OLS(D, Z_with_const).fit()
    
    # F 统计量
    # 对于单个工具变量，F = t^2
    t_stat = first_stage.tvalues[1]
    F_stat = t_stat ** 2
    
    # R^2
    r_squared = first_stage.rsquared
    
    return {
        'F_stat': F_stat,
        'r_squared': r_squared,
        'pi_hat': first_stage.params[1],
        'pi_se': first_stage.bse[1],
        't_stat': t_stat,
        'p_value': first_stage.pvalues[1]
    }

def iv_estimate(df):
    """
    2SLS 估计
    """
    # 使用 linearmodels
    df = df.copy()
    df['const'] = 1
    
    model = IV2SLS(df['Y'], df[['const']], df['D'], df['Z']).fit()
    
    return {
        'beta_hat': model.params['D'],
        'se': model.std_errors['D'],
        'ci_lower': model.params['D'] - 1.96 * model.std_errors['D'],
        'ci_upper': model.params['D'] + 1.96 * model.std_errors['D']
    }

# 诊断所有数据集
print("=" * 70)
print("弱工具变量诊断 (真实 β = 0.5)")
print("=" * 70)
print(f"\n{'IV 强度':<12} {'第一阶段 F':<15} {'π 估计':<12} {'2SLS β':<12} {'95% CI':<25}")
print("-" * 70)

datasets = [
    ('强', data_strong),
    ('中等', data_moderate),
    ('弱', data_weak),
    ('极弱', data_very_weak)
]

for name, df in datasets:
    diag = first_stage_diagnostics(df)
    est = iv_estimate(df)
    
    weak = '⚠️' if diag['F_stat'] < 10 else '✅'
    ci = f"[{est['ci_lower']:.3f}, {est['ci_upper']:.3f}]"
    
    print(f"{name:<12} {diag['F_stat']:<15.2f} {diag['pi_hat']:<12.3f} {est['beta_hat']:<12.3f} {ci:<25} {weak}")

In [ ]:
# 可视化 F 统计量与估计精度的关系
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 第一阶段散点图
for ax, (name, df) in zip(axes[0], [('强 IV', data_strong), ('弱 IV', data_weak)]):
    ax.scatter(df['Z'], df['D'], alpha=0.3, s=20)
    
    # 拟合线
    z = np.linspace(df['Z'].min(), df['Z'].max(), 100)
    diag = first_stage_diagnostics(df)
    d_hat = diag['pi_hat'] * z
    ax.plot(z, d_hat, 'r-', linewidth=2, label=f'π = {diag["pi_hat"]:.3f}')
    
    ax.set_xlabel('工具变量 Z')
    ax.set_ylabel('处理变量 D')
    ax.set_title(f'{name}: 第一阶段关系\nF = {diag["F_stat"]:.1f}')
    ax.legend()

# 模拟估计分布
def simulate_iv_distribution(iv_strength, n_sims=500, n_samples=1000):
    estimates = []
    for i in range(n_sims):
        df, _ = generate_iv_data(n=n_samples, iv_strength=iv_strength, seed=i)
        est = iv_estimate(df)
        estimates.append(est['beta_hat'])
    return np.array(estimates)

# 强 IV vs 弱 IV 的估计分布
print("模拟 IV 估计分布 (500 次模拟)...")
estimates_strong = simulate_iv_distribution('strong', n_sims=500)
estimates_weak = simulate_iv_distribution('weak', n_sims=500)

ax = axes[1, 0]
ax.hist(estimates_strong, bins=30, alpha=0.6, label='强 IV', color='#2ecc71')
ax.hist(estimates_weak, bins=30, alpha=0.6, label='弱 IV', color='#e74c3c')
ax.axvline(x=0.5, color='black', linestyle='--', linewidth=2, label='真实值')
ax.set_xlabel('β 估计值')
ax.set_ylabel('频数')
ax.set_title('强 IV vs 弱 IV 的估计分布')
ax.legend()

# 估计统计量
ax = axes[1, 1]
stats_data = {
    '强 IV': [np.mean(estimates_strong), np.std(estimates_strong), 
              np.mean(np.abs(estimates_strong - 0.5))],
    '弱 IV': [np.mean(estimates_weak), np.std(estimates_weak),
              np.mean(np.abs(estimates_weak - 0.5))]
}

x = np.arange(3)
width = 0.35
ax.bar(x - width/2, stats_data['强 IV'], width, label='强 IV', color='#2ecc71')
ax.bar(x + width/2, stats_data['弱 IV'], width, label='弱 IV', color='#e74c3c')
ax.set_xticks(x)
ax.set_xticklabels(['均值', '标准差', 'MAE'])
ax.set_ylabel('值')
ax.set_title('估计质量比较')
ax.legend()
ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

print(f"\n统计量:")
print(f"  强 IV: 均值={np.mean(estimates_strong):.3f}, 标准差={np.std(estimates_strong):.3f}")
print(f"  弱 IV: 均值={np.mean(estimates_weak):.3f}, 标准差={np.std(estimates_weak):.3f}")

### 2.2 Stock-Yogo 临界值

Stock & Yogo (2005) 提供了更精确的弱工具变量检验临界值。

In [ ]:
def weak_iv_test(df, max_bias=0.1):
    """
    弱工具变量检验
    
    Args:
        df: 数据
        max_bias: 可接受的最大相对偏差 (10% = 0.1)
    
    Returns:
        是否通过检验
    """
    diag = first_stage_diagnostics(df)
    F = diag['F_stat']
    
    # Stock-Yogo 临界值 (单个内生变量，单个工具变量)
    # 偏差阈值    10%   15%   20%   25%
    critical_values = {
        0.10: 16.38,
        0.15: 8.96,
        0.20: 6.66,
        0.25: 5.53
    }
    
    cv = critical_values.get(max_bias, 10)  # 默认用 10
    
    print("Stock-Yogo 弱工具变量检验")
    print("=" * 50)
    print(f"  第一阶段 F 统计量: {F:.2f}")
    print(f"  可接受相对偏差: {max_bias:.0%}")
    print(f"  临界值: {cv:.2f}")
    print()
    
    if F > cv:
        print(f"  ✅ F > {cv:.2f}，拒绝弱工具变量假设")
        return True
    else:
        print(f"  ❌ F < {cv:.2f}，不能拒绝弱工具变量假设")
        return False

# 检验
print("\n--- 强 IV 数据 ---")
weak_iv_test(data_strong, max_bias=0.10)

print("\n--- 弱 IV 数据 ---")
weak_iv_test(data_weak, max_bias=0.10)

---

## Part 3: 弱工具变量的稳健推断

### 3.1 Anderson-Rubin 置信区间

即使在弱工具变量情况下也保持正确覆盖率的方法。

In [ ]:
def anderson_rubin_ci(df, alpha=0.05, n_grid=100):
    """
    Anderson-Rubin 置信区间
    
    对于每个候选 β，检验 H0: β = β0
    """
    Y, D, Z = df['Y'].values, df['D'].values, df['Z'].values
    n = len(Y)
    
    # 搜索 β 的范围
    beta_range = np.linspace(-2, 3, n_grid)
    p_values = []
    
    for beta in beta_range:
        # 构造残差
        residual = Y - beta * D
        
        # 回归残差对 Z
        Z_with_const = sm.add_constant(Z)
        model = sm.OLS(residual, Z_with_const).fit()
        
        # F 检验
        f_stat = model.fvalue
        p_value = model.f_pvalue
        p_values.append(p_value)
    
    p_values = np.array(p_values)
    
    # 找到不被拒绝的区间
    in_ci = p_values > alpha
    
    if np.any(in_ci):
        ci_lower = beta_range[in_ci].min()
        ci_upper = beta_range[in_ci].max()
    else:
        ci_lower = np.nan
        ci_upper = np.nan
    
    return {
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'beta_range': beta_range,
        'p_values': p_values
    }

# 比较标准 CI 和 AR CI
print("=" * 60)
print("标准 2SLS CI vs Anderson-Rubin CI")
print("=" * 60)

for name, df in [('强 IV', data_strong), ('弱 IV', data_weak)]:
    est = iv_estimate(df)
    ar = anderson_rubin_ci(df)
    
    print(f"\n{name}:")
    print(f"  2SLS 估计: {est['beta_hat']:.3f}")
    print(f"  标准 CI:   [{est['ci_lower']:.3f}, {est['ci_upper']:.3f}] (宽度: {est['ci_upper']-est['ci_lower']:.3f})")
    print(f"  AR CI:     [{ar['ci_lower']:.3f}, {ar['ci_upper']:.3f}] (宽度: {ar['ci_upper']-ar['ci_lower']:.3f})")

In [ ]:
# 可视化 AR 置信区间
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, (name, df) in zip(axes, [('强 IV', data_strong), ('弱 IV', data_weak)]):
    ar = anderson_rubin_ci(df, n_grid=200)
    
    ax.plot(ar['beta_range'], ar['p_values'], 'b-', linewidth=2)
    ax.axhline(y=0.05, color='red', linestyle='--', label='α = 0.05')
    ax.axvline(x=0.5, color='green', linestyle='--', label='真实 β = 0.5')
    
    # 标记 CI
    ax.fill_between(ar['beta_range'], 0, ar['p_values'], 
                   where=ar['p_values'] > 0.05, alpha=0.3, color='blue',
                   label=f'95% CI: [{ar["ci_lower"]:.2f}, {ar["ci_upper"]:.2f}]')
    
    ax.set_xlabel('β')
    ax.set_ylabel('p-value')
    ax.set_title(f'{name}: Anderson-Rubin 检验')
    ax.legend()
    ax.set_ylim(0, 1)

plt.tight_layout()
plt.show()

### 3.2 LIML 估计

Limited Information Maximum Likelihood 在弱工具变量下比 2SLS 更稳健。

In [ ]:
def liml_estimate(df):
    """
    LIML 估计
    
    比 2SLS 更稳健的 IV 估计方法
    """
    from linearmodels.iv import IVLIML
    
    df = df.copy()
    df['const'] = 1
    
    model = IVLIML(df['Y'], df[['const']], df['D'], df['Z']).fit()
    
    return {
        'beta_hat': model.params['D'],
        'se': model.std_errors['D'],
        'ci_lower': model.params['D'] - 1.96 * model.std_errors['D'],
        'ci_upper': model.params['D'] + 1.96 * model.std_errors['D']
    }

# 比较 2SLS 和 LIML
print("=" * 60)
print("2SLS vs LIML 比较")
print("=" * 60)

for name, df in datasets:
    est_2sls = iv_estimate(df)
    est_liml = liml_estimate(df)
    
    print(f"\n{name} IV:")
    print(f"  2SLS: β = {est_2sls['beta_hat']:.3f} (SE = {est_2sls['se']:.3f})")
    print(f"  LIML: β = {est_liml['beta_hat']:.3f} (SE = {est_liml['se']:.3f})")

---

## Part 4: 排斥性假设违背

### 4.1 问题描述

排斥性假设：工具变量 $Z$ 只通过处理 $D$ 影响结果 $Y$。

如果 $Z$ 直接影响 $Y$，IV 估计有偏。

In [ ]:
def generate_invalid_iv_data(n=1000, beta_true=0.5, direct_effect=0.3, seed=42):
    """
    生成违反排斥性假设的 IV 数据
    
    Z 直接影响 Y（不仅通过 D）
    """
    np.random.seed(seed)
    
    U = np.random.randn(n)
    Z = np.random.randn(n)
    
    D = 0.5 * Z + 0.5 * U + np.random.randn(n) * 0.5
    
    # Y 受 D、U 和 Z（直接效应）影响
    Y = beta_true * D + 0.8 * U + direct_effect * Z + np.random.randn(n) * 0.5
    
    return pd.DataFrame({'Y': Y, 'D': D, 'Z': Z})

# 生成无效 IV 数据
data_invalid = generate_invalid_iv_data(direct_effect=0.3)

# 估计
est_invalid = iv_estimate(data_invalid)
diag_invalid = first_stage_diagnostics(data_invalid)

print("排斥性假设违背示例")
print("=" * 50)
print(f"真实 β = 0.5, Z 的直接效应 = 0.3")
print(f"\n2SLS 估计: {est_invalid['beta_hat']:.3f}")
print(f"第一阶段 F: {diag_invalid['F_stat']:.1f}")
print(f"\n⚠️ 估计严重偏离真实值!")
print(f"   偏差来源: Z 直接影响 Y，违反排斥性")

### 4.2 Sargan-Hansen 过度识别检验

当有多个工具变量时，可以检验排斥性假设。

In [ ]:
def generate_multiple_iv_data(n=1000, beta_true=0.5, invalid_iv=False, seed=42):
    """
    生成多工具变量数据
    """
    np.random.seed(seed)
    
    U = np.random.randn(n)
    Z1 = np.random.randn(n)
    Z2 = np.random.randn(n)
    
    D = 0.4 * Z1 + 0.3 * Z2 + 0.5 * U + np.random.randn(n) * 0.5
    
    if invalid_iv:
        # Z2 直接影响 Y（无效）
        Y = beta_true * D + 0.8 * U + 0.4 * Z2 + np.random.randn(n) * 0.5
    else:
        Y = beta_true * D + 0.8 * U + np.random.randn(n) * 0.5
    
    return pd.DataFrame({'Y': Y, 'D': D, 'Z1': Z1, 'Z2': Z2})

def overidentification_test(df):
    """
    Sargan-Hansen 过度识别检验
    
    H0: 所有工具变量都是有效的
    """
    df = df.copy()
    df['const'] = 1
    
    model = IV2SLS(df['Y'], df[['const']], df['D'], df[['Z1', 'Z2']]).fit()
    
    # Sargan 检验统计量
    sargan_stat = model.sargan.stat
    sargan_pval = model.sargan.pval
    
    return {
        'sargan_stat': sargan_stat,
        'p_value': sargan_pval,
        'beta_hat': model.params['D'],
        'se': model.std_errors['D']
    }

# 有效 IV
data_valid_multi = generate_multiple_iv_data(invalid_iv=False)
test_valid = overidentification_test(data_valid_multi)

# 无效 IV
data_invalid_multi = generate_multiple_iv_data(invalid_iv=True)
test_invalid = overidentification_test(data_invalid_multi)

print("Sargan-Hansen 过度识别检验")
print("=" * 50)
print("H0: 所有工具变量都是有效的")

print(f"\n有效 IV 情况:")
print(f"  Sargan 统计量: {test_valid['sargan_stat']:.3f}")
print(f"  p-value: {test_valid['p_value']:.4f}")
print(f"  结论: {'✅ 不拒绝 H0' if test_valid['p_value'] > 0.05 else '❌ 拒绝 H0'}")

print(f"\n无效 IV 情况 (Z2 直接影响 Y):")
print(f"  Sargan 统计量: {test_invalid['sargan_stat']:.3f}")
print(f"  p-value: {test_invalid['p_value']:.4f}")
print(f"  结论: {'✅ 不拒绝 H0' if test_invalid['p_value'] > 0.05 else '❌ 拒绝 H0，存在无效 IV'}")

---

## Part 5: 完整诊断流程

In [ ]:
def iv_diagnostic_pipeline(df, iv_cols=['Z'], alpha=0.05):
    """
    完整的 IV 诊断流程
    """
    print("=" * 70)
    print("工具变量诊断报告")
    print("=" * 70)
    
    # Step 1: 第一阶段诊断
    print("\n【Step 1: 第一阶段诊断】")
    diag = first_stage_diagnostics(df)
    print(f"  π (IV → D): {diag['pi_hat']:.4f} (SE = {diag['pi_se']:.4f})")
    print(f"  t 统计量: {diag['t_stat']:.2f}")
    print(f"  F 统计量: {diag['F_stat']:.2f}")
    
    # Step 2: 弱 IV 检验
    print("\n【Step 2: 弱工具变量检验】")
    if diag['F_stat'] > 16.38:
        print(f"  ✅ F > 16.38，IV 足够强（偏差 < 10%）")
        weak_iv = False
    elif diag['F_stat'] > 10:
        print(f"  ⚡ 10 < F < 16.38，IV 中等强度")
        weak_iv = False
    else:
        print(f"  ❌ F < 10，存在弱工具变量问题")
        weak_iv = True
    
    # Step 3: 估计
    print("\n【Step 3: 参数估计】")
    est_2sls = iv_estimate(df)
    print(f"  2SLS 估计: {est_2sls['beta_hat']:.4f}")
    print(f"  标准 CI: [{est_2sls['ci_lower']:.4f}, {est_2sls['ci_upper']:.4f}]")
    
    if weak_iv:
        ar = anderson_rubin_ci(df)
        print(f"  AR CI: [{ar['ci_lower']:.4f}, {ar['ci_upper']:.4f}] (对弱 IV 稳健)")
    
    # Step 4: 建议
    print("\n【Step 4: 建议】")
    if weak_iv:
        print("  ⚠️ 检测到弱工具变量:")
        print("     1. 使用 Anderson-Rubin 置信区间")
        print("     2. 考虑 LIML 估计")
        print("     3. 寻找更强的工具变量")
    else:
        print("  ✅ 工具变量足够强，可使用标准 2SLS")
    
    return {
        'weak_iv': weak_iv,
        'F_stat': diag['F_stat'],
        'beta_2sls': est_2sls['beta_hat']
    }

# 运行诊断
print("\n" + "#" * 70)
print("场景: 弱工具变量")
print("#" * 70)
iv_diagnostic_pipeline(data_weak)

---

## 💡 思考题

1. **多弱工具变量**：如果有多个弱工具变量，能否"加总"成一个强工具？
   - 提示：考虑 JIVE 估计

2. **局部处理效应**：IV 估计的是什么效应？与 ATE 有何区别？
   - 提示：LATE (Local Average Treatment Effect)

3. **工具变量选择**：如何判断一个变量是否适合做工具变量？
   - 提示：需要领域知识，统计检验只能检验相关性

---

## 📝 总结

### 弱 IV 诊断规则

| F 统计量 | 诊断 | 建议 |
|---------|------|------|
| F > 16.38 | 强 IV | 标准 2SLS |
| 10 < F < 16.38 | 中等 | 谨慎使用，报告 AR CI |
| F < 10 | 弱 IV | 使用 AR/LIML，或寻找更好的 IV |

### 弱 IV 稳健方法

| 方法 | 优点 | 缺点 |
|------|------|------|
| Anderson-Rubin | 正确覆盖率 | CI 可能很宽 |
| LIML | 偏差更小 | 方差可能更大 |
| JIVE | 适合多 IV | 计算复杂 |

### 面试要点

- 能解释工具变量的两个条件
- 知道 F > 10 规则
- 了解弱 IV 的稳健方法
- 理解过度识别检验的作用